In [2]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=1000)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def listmap(f,lst):return list(map(f,lst))

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay")

okay


In [8]:
for n in [1..5]:
    print((n,sigma(n,k=11)))

(1, 1)
(2, 2049)
(3, 177148)
(4, 4196353)
(5, 48828126)


In [ ]:
import time

def Sigma(n):
    ans=0
    for a in [1..n+1]:
        term=sigma(a,k=1) 
        ans=ans+term*x^a
    return D(ans)


def recipSigma(n):
    g=L(1/Sigma(2*n))
    return g

def recipSigmaPowerConstant(n):
    f=recipSigma(n)^n
    return f[0]


L=LaurentSeriesRing(QQ,'x', default_prec=5100)
import pickle
datafile= open('/Users/barrybrent/run10may23.txt','wb') 
data=[]
start=time.time()/60
for a in [1..5000]:
    c=recipSigmaPowerConstant(a)
    data=data+[(a,c)]
    finish=time.time()/60
    print((a,finish-start))
d = pickle.dumps(str(data))
datafile.write(d)
datafile.close()
#-----------------------------------------------------

import time

def Sigma(n):
    ans=0
    for a in [1..n+1]:
        term=sigma(a,k=1) 
        ans=ans+term*x^a
    return D(ans)


def recipSigma(n):
    g=L(1/Sigma(2*n))
    return g

def recipSigmaPowerConstant(n):
    f=recipSigma(n)^n
    return f[0]


L=LaurentSeriesRing(QQ,'x', default_prec=10000)
import pickle
datafile= open('/Users/barrybrent/run10may23no2.txt','wb') 
data=[]
start=time.time()/60
for a in [5001..7000]:
    c=recipSigmaPowerConstant(a)
    data=data+[(a,c)]
    finish=time.time()/60
    print((a,finish-start))
d = pickle.dumps(str(data))
datafile.write(d)
datafile.close()

In [3]:
a=[[1],[2]];b=[[3],[4]];c=a+b;print(c);
data=[]
for tpl in c:data=data+tpl
print(data)

[[1], [2], [3], [4]]
[1, 2, 3, 4]


In [6]:
import pickle
rfile1=open('/Users/barrybrent/run10may23.txt','rb') 
rs1=pickle.load(rfile1)
rfile1.close
s1=stripQuotationMarks(rs1)

rfile2=open('/Users/barrybrent/run10may23no2.txt','rb') 
rs2=pickle.load(rfile2)
rfile2.close
s2=stripQuotationMarks(rs2)

datafile= open('/Users/barrybrent/run12may23.txt','wb') 
data=[]
s=s1+s2
for tpl in s:
    data=data+[tpl] # contrary to the cell above, Sage insisted on the brackets in this one.
d = pickle.dumps(str(data))
datafile.write(d)
datafile.close()

In [7]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)
print(s[0]);print(s[1]);
print(len(s))


(1, -3)
(2, 19)
7000


In [3]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

yes=0
no=0
for k in [0..6999]:
    pair=s[k]
    a=pair[0]
    c=pair[1]
    o2=order(2,c)
    if o2==0:yes=yes+1
    if o2!=0:n=no+1
print((yes,no))

(7000, 0)


In [4]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

yes=0
no=0
for k in [0..6999]:
    pair=s[k]
    a=pair[0]
    if mod(a,2)==0:
        c=pair[1]
        o3=order(3,c)
        if o3==0:yes=yes+1
        if o3!=0:no=no+1
print((yes,no)) 

(3500, 0)


In [5]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

yes=0
no=0
for k in [0..6999]:
    pair=s[k]
    a=pair[0]
    c=pair[1]
    o3=order(3,c)
    if o3==0:
        if mod(a,2)==0:
            yes=yes+1
        if mod(a,2)!=0:
            no=no+1
print((yes,no))

(3500, 0)


In [9]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

yes=[]
no=[]
for k in [0..699]:
    pair=s[k]
    a=pair[0]
    if mod(a,6)==3:
        c=pair[1]
        o3=order(3,c)
        if o3==1:yes=yes+[a]
        if o3!=1:no=no+[a]
print((len(yes),len(no)))

(117, 0)


In [10]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

yes=[]
no=[]
for k in [0..699]:
    pair=s[k]
    a=pair[0]
    c=pair[1]
    o3=order(3,c)
    if  o3==1:
        if ZZ(mod(a,6))==3:
            yes=yes+[a]
        if ZZ(mod(a,6))!=3:no=no+[a]
print((len(yes),len(no)))

(117, 193)


In [6]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

yes=0
no=0
for k in [0..6999]:
    pair=s[k]
    a=pair[0]
    if mod(a,6)==1:
        c=pair[1]
        o3=order(3,c)
        if o3==1:yes=yes+1
        if o3!=1:no=no+1
print((yes,no))
        

(1167, 0)


In [12]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

yes=[]
no=[]
for k in [0..699]:
    pair=s[k]
    a=pair[0]
    c=pair[1]
    o3=order(3,c)
    if  o3==1:
        if ZZ(mod(a,6))==1:
            yes=yes+[a]
        if ZZ(mod(a,6))!=3:no=no+[a]
print((len(yes),len(no)))

(117, 193)


In [13]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

yes=0
no=0
for k in [0..6999]:
    pair=s[k]
    a=pair[0]
    if mod(a,6)==5:
        c=pair[1]
        o3=order(3,c)
        if o3==1:yes=yes+1
        if o3!=1:no=no+1
print((yes,no))

(802, 364)


In [11]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

yes=[]
no=[]
for k in [0..6999]:
    pair=s[k]
    a=pair[0]
    if mod(a,2)==0:
        c=pair[1]
        o3=order(3,c)
        sk=c/3^o3
        m3=mod(sk,3)
        if m3==1:yes=yes+[a]
        if m3!=1:no=no+[a]
print((len(yes),len(no)))

(3500, 0)


In [14]:
import pickle
rfile=open('/Users/barrybrent/run12may23.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)

yes=[]
no=[]
for k in [0..6999]:
    pair=s[k]
    a=pair[0]
    if mod(a,2)==1:
        c=pair[1]
        o3=order(3,c)
        sk=c/3^o3
        m3=mod(sk,3)
        if m3==1:yes=yes+[a]
        if m3!=1:no=no+[a]
print((len(yes),len(no)))

(543, 2957)
